# 126.33: Steady state after step head change.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ttim

from bruggeman.flow1d import bruggeman_126_33

View the function.

_(`lambda_` is used to avoid problems with `latexify-py` and `Jupyter-notebook`)_

In [ ]:
bruggeman_126_33

View the docstring to get a description of the input parameters

In [ ]:
help(bruggeman_126_33)

Define some aquifer parameters.

In [ ]:
k = 20.0  # hydraulic conductivity, m/d
D = 50.0  # thickness of aquifer, m
T = k * D  # transmissivity, m^2/d
c = 1000  # leakage factor, d
w = 20  # entry resistance at x=0, d
h_step = 1.0  # head at x=0, m
S = 0.001  # storage coefficient of aquifer, [-]

Saq = S / D  # specific storage [1/m]
Sll = 0.001  # specific storage of leaky layer, [-]

Set up a `ttim` model. 

The ModelXsection to be able to plot the sections, but the same result could be reached with the 'normal' ModelMaq model.

In [ ]:
# The model
ml = ttim.ModelXsection(naq=1, tmin=1e-4, tmax=1e3)

riv = ttim.XsectionMaq(
    model=ml,
    x1=-np.inf,
    x2=0,
    kaq=k,
    z=[0, -D],
    Saq=1e-20,
    Sll=1e-20,
    topboundary="confined",
    phreatictop=False,
    name="river",
)

land = ttim.XsectionMaq(
    model=ml,
    x1=0,
    x2=np.inf,
    kaq=k,
    z=[5, 0, -D],
    c=c,
    Saq=Saq,
    Sll=Sll,
    topboundary="semi",
    phreatictop=False,
    name="hinterland",
)

# Use a small offset to avoid a singular matrix.
small = 1e-5

river_hls = ttim.HeadLineSink1D(
    model=ml,
    xls=0 - small,
    tsandh=[0, h_step],
    res=w,
)

ml.solve()

ax = riv.plot(params=True, names=True, labels=False)
land.plot(ax=ax, params=True, names=True, labels=False)
river_hls.plot(ax=ax)

ax.set_xlim(-100, 100)
ax.set_ylim(ymax=12)

Compare `ttim` implementation to the analytical solution from Bruggeman.

In [ ]:
x = np.linspace(0, 1000, 101)
y = np.zeros_like(x)
t = np.logspace(-1, 1, 3)

plt.figure(figsize=(10, 3))
for i in range(len(t)):
    h = ml.headalongline(x, y, t[i])
    plt.plot(x, h.squeeze(), label=f"t={t[i]:.2f} d")
    ha = bruggeman_126_33(x, h_step, k, D, c, w)
    plt.plot(x, ha, "k:")
plt.plot([], [], "k:", label="Bruggeman 126.33")
plt.legend(loc=(0, 1), frameon=False, ncol=6, fontsize="small")
plt.xlabel("x [m]")
plt.ylabel("drawdown [m]")
plt.grid()
plt.ylim(0, h_step * 1.2)
plt.axvline(0, color="grey", linewidth=3)
plt.plot([-50, 0], [h_step, h_step], "b")
plt.xlim(-50, 1000)

As can be seen from the graph, the `storativity` causes a delay in reaching the steady-state value. When the storativity approaches zero, the steady-state is immediately reached.

Furthermore, the head in the aquifer never reaches the head in the river, because of the entry resistance.